In [1]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
import talib
import ccxt
from configparser import ConfigParser
from binance.client import Client
from binance.enums import *
from utils.helpers import check_decimals, round_up

In [2]:
CREDENTIALS_PATH = os.path.join(os.path.expanduser('~'), ".binance/credentials")
config = ConfigParser()
config.read(CREDENTIALS_PATH)
api_key = config.get('trade', 'API_KEY')
api_secret  = config.get('trade', 'SECRET_KEY')

In [3]:
# Initialize Variables
CANDLE_DURATION_IN_MIN = 30

RSI_PERIOD = 14
RSI_OVERBOUGHT = 55
RSI_OVERSOLD = 45

CCXT_TICKER_NAME = 'DOGE/USDT'
TRADING_TICKER_NAME = 'DOGEUSDT'

INVESTMENT_AMOUNT_DOLLARS = 10
HOLDING_QUANTITY = 0

exchange = ccxt.binance()
binance_client = Client(api_key=api_key, api_secret=api_secret)
DECIMALS_QUANTITY = check_decimals(TRADING_TICKER_NAME, binance_client)

In [4]:
10/88.13

0.11346873936230568

In [5]:
# df = pd.DataFrame(binance_client.get_all_tickers())
# current_price = float(df[df.symbol == "MANAUSDT"]['price'])
# current_price

In [6]:
# STEP 1: FETCH THE DATA
def fetch_data(ticker):
    global exchange
    bars,ticker_df = None, None

    try:
        bars = exchange.fetch_ohlcv(ticker, timeframe=f'{CANDLE_DURATION_IN_MIN}m', limit=100)
    except:
        print(f"Error in fetching data from the exchange:{ticker}")

    if bars is not None:
        ticker_df = pd.DataFrame(bars[:-1], columns=['at', 'open', 'high', 'low', 'close', 'vol'])
        ticker_df['Date'] = pd.to_datetime(ticker_df['at'], unit='ms')
        ticker_df['symbol'] = ticker

    return ticker_df

In [7]:
# STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
def get_trade_recommendation(ticker_df):

    macd_result = 'WAIT'
    final_result = 'WAIT'

    # BUY or SELL based on MACD crossover points and the RSI value at that point
    macd, signal, hist = talib.MACD(ticker_df['close'], fastperiod = 12, slowperiod = 26, signalperiod = 9)
    last_hist = hist.iloc[-1]
    prev_hist = hist.iloc[-2]
    if not np.isnan(prev_hist) and not np.isnan(last_hist):
        # If hist value has changed from negative to positive or vice versa, it indicates a crossover
        macd_crossover = (abs(last_hist + prev_hist)) != (abs(last_hist) + abs(prev_hist))
        if macd_crossover:
            macd_result = 'BUY' if last_hist > 0 else 'SELL'

    if macd_result != 'WAIT':
        rsi = talib.RSI(ticker_df['close'], timeperiod = 14)
        # Consider last 3 RSI values
        last_rsi_values = rsi.iloc[-3:]

        if (last_rsi_values.min() <= RSI_OVERSOLD):
            final_result = 'BUY'
        elif (last_rsi_values.max() >= RSI_OVERBOUGHT):
            final_result = 'SELL'
    print("MACD Result:", macd_result, "Final Result:", final_result)
    return final_result if final_result == macd_result else 'WAIT'

In [8]:
# STEP 3: EXECUTE THE TRADE
def execute_trade(trade_rec_type, trading_ticker):
    global binance_client, HOLDING_QUANTITY, DECIMALS_QUANTITY
    order_placed = False
    side_value = SIDE_BUY if (trade_rec_type == "BUY") else SIDE_SELL
    try:
        df = pd.DataFrame(binance_client.get_all_tickers())
        df = df[df.symbol == trading_ticker]
        if True:
            current_price = float(df['price'])
            script_quantity = round_up(INVESTMENT_AMOUNT_DOLLARS/current_price, DECIMALS_QUANTITY) if trade_rec_type == "BUY" else HOLDING_QUANTITY
            print(f"PLACING ORDER {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}: "
                  f"{trading_ticker}, {side_value}, {current_price}, {script_quantity}, {int(time.time() * 1000)} ")
            
            order_response = binance_client.create_order(symbol=trading_ticker,
                                                         side = side_value,
                                                         type=ORDER_TYPE_LIMIT,
                                                         timeInForce = TIME_IN_FORCE_GTC,
                                                         quantity=script_quantity,
                                                         price=str(current_price))
                        
            print(f"ORDER PLACED")
            HOLDING_QUANTITY = script_quantity if trade_rec_type == "BUY" else HOLDING_QUANTITY
            order_placed = True

    except:
        print(f"\nALERT!!! UNABLE TO COMPLETE ORDER")
    
    return order_placed


In [9]:
# STEP 3: EXECUTE THE TRADE
def execute_trade2(trade_rec_type, trading_ticker):
    global binance_client, HOLDING_QUANTITY, DECIMALS_QUANTITY
    order_placed = False
    side_value = SIDE_BUY if (trade_rec_type == "BUY") else SIDE_SELL

    df = pd.DataFrame(binance_client.get_all_tickers())
    df = df[df.symbol == trading_ticker]
    if True:
        current_price = float(df['price'])
        script_quantity = round_up(INVESTMENT_AMOUNT_DOLLARS/current_price, DECIMALS_QUANTITY) if trade_rec_type == "BUY" else HOLDING_QUANTITY
        print(f"PLACING ORDER {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}: "
                f"{trading_ticker}, {side_value}, {current_price}, {script_quantity}, {int(time.time() * 1000)} ")
        
        order_response = binance_client.create_order(symbol=trading_ticker,
                                                        side = side_value,
                                                        type=ORDER_TYPE_LIMIT,
                                                        timeInForce = TIME_IN_FORCE_GTC,
                                                        quantity=script_quantity,
                                                        price=str(current_price))
                    
        print(f"ORDER PLACED")
        HOLDING_QUANTITY = script_quantity if trade_rec_type == "BUY" else HOLDING_QUANTITY
        order_placed = True

    return order_placed

arrumar questão do currently_holding, criar stop loss, após atingir a alta 

In [10]:
def run_bot_for_ticker(ccxt_ticker, trading_ticker):

    currently_holding = False
    while 1:
        # STEP 1: FETCH THE DATA
        ticker_data = fetch_data(ccxt_ticker)
        if ticker_data is not None:
            # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
            trade_rec_type = get_trade_recommendation(ticker_data)
            print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} TRADING RECOMMENDATION: {trade_rec_type}')

            # STEP 3: EXECUTE THE TRADE
            if (trade_rec_type == 'BUY' and not currently_holding) or \
                (trade_rec_type == 'SELL' and currently_holding):
                print(f'Placing {trade_rec_type} order')
                trade_successful = execute_trade2(trade_rec_type,trading_ticker)
                currently_holding = not currently_holding if trade_successful else currently_holding

            time.sleep(CANDLE_DURATION_IN_MIN*60)
        else:
            print(f'Unable to fetch ticker data - {ccxt_ticker}. Retrying!!')
            time.sleep(5)


In [11]:
run_bot_for_ticker(CCXT_TICKER_NAME,TRADING_TICKER_NAME)

MACD Result: WAIT Final Result: WAIT
28/04/2022 12:07:34 TRADING RECOMMENDATION: WAIT


In [ ]:
0.11*88.98

In [ ]:
trading_ticker = "LUNAUSDT"
side_value = SIDE_BUY
script_quantity = 0.12
current_price = 88.98

In [15]:
order_response = binance_client.create_order(symbol=trading_ticker,
                                                        side = side_value,
                                                        type=ORDER_TYPE_LIMIT,
                                                        timeInForce = TIME_IN_FORCE_GTC,
                                                        quantity=script_quantity,
                                                        price=str(current_price))

In [28]:
trading_ticker = 'LUNAUSDT'
script_quantity = 0.13
current_price = 80.62
side_value = SIDE_BUY

In [12]:
11/96.21

0.1143332293940339

In [13]:
10/96.21

0.10393929944912172

In [20]:
0.12*96.21

11.5452

In [11]:
0.62*18.02

11.1724

In [29]:
order = binance_client.create_test_order(
    symbol=trading_ticker,
    side=SIDE_BUY,
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_GTC,
    quantity=script_quantity,
    price=str(current_price))

In [30]:
order_response = binance_client.create_order(symbol=trading_ticker,
                                                         side = side_value,
                                                         type=ORDER_TYPE_LIMIT,
                                                         timeInForce = TIME_IN_FORCE_GTC,
                                                         quantity=script_quantity,
                                                         price=str(current_price))

In [34]:
binance_client.cancel_order(symbol = 'LUNAUSDT', orderId = 1602345537)

{'symbol': 'LUNAUSDT',
 'origClientOrderId': '8Pu9K6weScKQPTzx3xMOOY',
 'orderId': 1602345537,
 'orderListId': -1,
 'clientOrderId': 'UZs1HQsjCzNJ25rZwznObG',
 'price': '80.62000000',
 'origQty': '0.13000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'CANCELED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY'}

In [19]:
check_decimals("CAKEUSDT", binance_client)

2

In [9]:
binance_client.get_symbol_info("CAKEUSDT")

{'symbol': 'CAKEUSDT',
 'status': 'TRADING',
 'baseAsset': 'CAKE',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.01000000',
   'maxPrice': '10000.00000000',
   'tickSize': '0.01000000'},
  {'filterType': 'PERCENT_PRICE',
   'multiplierUp': '5',
   'multiplierDown': '0.2',
   'avgPriceMins': 5},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.01000000',
   'maxQty': '90000.00000000',
   'stepSize': '0.01000000'},
  {'filterType': 'MIN_NOTIONAL',
   'minNotional': '10.00000000',
   'applyToMarket': True,
   'avgPriceMins': 5},
  {'filterType': 'ICE